In [ ]:
"""
=================================================================================
TROPICAL DISEASE PREDICTION DATA COLLECTION SYSTEM
Central Africa Focus: DRC, Congo, CAR, Cameroon
=================================================================================

Author: Research Team
Date: 2025
Platform: Google Colab
Purpose: Automated data collection for ICICT 2026 paper

Data Sources:
1. Our World in Data (OWID) - Malaria statistics
2. WHO Global Health Observatory (GHO) - Disease data via API
3. NASA POWER - Climate data
4. World Bank - Socioeconomic indicators

Output: Master dataset ready for machine learning
=================================================================================
"""

# =============================================================================
# SECTION 1: INSTALLATION & IMPORTS
# =============================================================================

print("=" * 80)
print("SECTION 1: Installing Required Libraries")
print("=" * 80)

# Install required packages
!pip install -q wbdata pandas numpy matplotlib seaborn requests openpyxl

# Import libraries
import pandas as pd
import numpy as np
import requests
import time
import os
from datetime import datetime
from io import BytesIO
import matplotlib.pyplot as plt
import seaborn as sns
import json

print("✅ All libraries imported successfully!\n")

# =============================================================================
# SECTION 2: CONFIGURATION
# =============================================================================

print("=" * 80)
print("SECTION 2: Configuration Setup")
print("=" * 80)

# Configuration dictionary
CONFIG = {
    'countries_iso': ['COD', 'COG', 'CAF', 'CMR'],  # DRC, Congo, CAR, Cameroon
    'country_names': {
        'COD': 'DRC',
        'COG': 'Congo',
        'CAF': 'CAR',
        'CMR': 'Cameroon'
    },
    'start_year': 2010,
    'end_year': 2024,
    'output_dir': '/content/data',
    'plots_dir': '/content/plots'
}

# Create output directories
os.makedirs(CONFIG['output_dir'], exist_ok=True)
os.makedirs(CONFIG['plots_dir'], exist_ok=True)

# Cities for climate data (major cities in each country)
CITIES = {
    # DRC
    'Kinshasa': {'lat': -4.3276, 'lon': 15.3136, 'country': 'DRC'},
    'Lubumbashi': {'lat': -11.6667, 'lon': 27.4667, 'country': 'DRC'},
    'Mbuji_Mayi': {'lat': -6.1500, 'lon': 23.6000, 'country': 'DRC'},
    'Kisangani': {'lat': 0.5167, 'lon': 25.2000, 'country': 'DRC'},
    'Goma': {'lat': -1.6792, 'lon': 29.2228, 'country': 'DRC'},

    # Congo
    'Brazzaville': {'lat': -4.2634, 'lon': 15.2429, 'country': 'Congo'},
    'Pointe_Noire': {'lat': -4.7692, 'lon': 11.8636, 'country': 'Congo'},

    # CAR
    'Bangui': {'lat': 4.3947, 'lon': 18.5582, 'country': 'CAR'},

    # Cameroon
    'Yaounde': {'lat': 3.8480, 'lon': 11.5021, 'country': 'Cameroon'},
    'Douala': {'lat': 4.0511, 'lon': 9.7679, 'country': 'Cameroon'}
}

print("✅ Configuration complete!")
print(f"   Countries: {list(CONFIG['country_names'].values())}")
print(f"   Period: {CONFIG['start_year']} - {CONFIG['end_year']}")
print(f"   Cities: {len(CITIES)} major cities")
print(f"   Output directory: {CONFIG['output_dir']}\n")

# =============================================================================
# SECTION 3: DATA COLLECTION FUNCTIONS
# =============================================================================

print("=" * 80)
print("SECTION 3: Defining Data Collection Functions")
print("=" * 80)

# -----------------------------------------------------------------------------
# 3.1 Our World in Data (OWID) - Malaria Data
# -----------------------------------------------------------------------------

def download_owid_malaria_data():
    """
    Download malaria data from Our World in Data
    This is the most reliable source with WHO-aggregated data

    Returns:
        DataFrame: Malaria deaths and incidence data
    """

    print("\n📥 Downloading malaria data from Our World in Data...")

    datasets = {}

    # URLs for different OWID malaria datasets
    urls = {
        'deaths': 'https://raw.githubusercontent.com/owid/owid-datasets/master/datasets/Malaria%20deaths%20by%20age%20-%20IHME%2C%20Global%20Burden%20of%20Disease/Malaria%20deaths%20by%20age%20-%20IHME%2C%20Global%20Burden%20of%20Disease.csv',
        'incidence': 'https://raw.githubusercontent.com/owid/owid-datasets/master/datasets/Malaria%20-%20World%20Health%20Organization%20(2023)/Malaria%20-%20World%20Health%20Organization%20(2023).csv'
    }

    # Country name mapping for OWID
    country_mapping = {
        'Democratic Republic of Congo': 'DRC',
        'Congo': 'Congo',
        'Central African Republic': 'CAR',
        'Cameroon': 'Cameroon'
    }

    for dataset_name, url in urls.items():
        try:
            df = pd.read_csv(url)
            print(f"   ✅ Downloaded {dataset_name}: {len(df)} total records")

            # Filter for target countries
            target_countries = list(country_mapping.keys())
            mask = df['Entity'].isin(target_countries)
            filtered_df = df[mask].copy()

            # Filter for years
            filtered_df = filtered_df[
                (filtered_df['Year'] >= CONFIG['start_year']) &
                (filtered_df['Year'] <= CONFIG['end_year'])
            ]

            # Standardize country names
            filtered_df['Country'] = filtered_df['Entity'].map(country_mapping)

            # Save
            filename = f"{CONFIG['output_dir']}/OWID_Malaria_{dataset_name.title()}.csv"
            filtered_df.to_csv(filename, index=False)

            datasets[dataset_name] = filtered_df
            print(f"      → Filtered: {len(filtered_df)} records for Central Africa")
            print(f"      → Saved: {filename}")

        except Exception as e:
            print(f"   ❌ Error downloading {dataset_name}: {e}")
            datasets[dataset_name] = None

    return datasets

# -----------------------------------------------------------------------------
# 3.2 WHO GHO API - Malaria Indicators
# -----------------------------------------------------------------------------

def download_who_gho_data():
    """
    Download malaria data from WHO Global Health Observatory API

    Returns:
        DataFrame: Combined WHO malaria indicators
    """

    print("\n📥 Downloading WHO GHO malaria data via API...")

    base_url = "https://ghoapi.azureedge.net/api"

    # WHO malaria indicators
    indicators = {
        'MALARIA_EST_CASES': 'Estimated_Malaria_Cases',
        'MALARIA_EST_DEATHS': 'Estimated_Malaria_Deaths',
        'MALARIA_CONF_CASES': 'Confirmed_Malaria_Cases',
        'MALARIA_INCD': 'Malaria_Incidence_per_1000',
        'MALARIA_MORT': 'Malaria_Mortality_per_100000'
    }

    all_data = []

    for indicator_code, indicator_name in indicators.items():
        print(f"\n   Processing: {indicator_name}")

        for country_code in CONFIG['countries_iso']:
            try:
                url = f"{base_url}/{indicator_code}"
                params = {
                    '$filter': f"SpatialDim eq '{country_code}'",
                    '$format': 'json'
                }

                response = requests.get(url, params=params, timeout=30)

                if response.status_code == 200:
                    data = response.json()

                    if 'value' in data and len(data['value']) > 0:
                        for record in data['value']:
                            year = record.get('TimeDim')
                            if year and CONFIG['start_year'] <= int(year) <= CONFIG['end_year']:
                                all_data.append({
                                    'Country_Code': country_code,
                                    'Country': CONFIG['country_names'][country_code],
                                    'Year': int(year),
                                    'Indicator': indicator_name,
                                    'Value': record.get('NumericValue'),
                                    'Low_Estimate': record.get('Low'),
                                    'High_Estimate': record.get('High')
                                })

                        count = len([r for r in data['value']
                                   if CONFIG['start_year'] <= int(r.get('TimeDim', 0)) <= CONFIG['end_year']])
                        print(f"      ✅ {CONFIG['country_names'][country_code]}: {count} records")
                    else:
                        print(f"      ⚠️  {CONFIG['country_names'][country_code]}: No data available")
                else:
                    print(f"      ❌ {CONFIG['country_names'][country_code]}: HTTP {response.status_code}")

            except Exception as e:
                print(f"      ❌ {CONFIG['country_names'][country_code]}: Error - {e}")

            time.sleep(0.5)  # Rate limiting

    if all_data:
        df = pd.DataFrame(all_data)

        # Create pivot table for easier use
        pivot_df = df.pivot_table(
            index=['Country', 'Country_Code', 'Year'],
            columns='Indicator',
            values='Value',
            aggfunc='first'
        ).reset_index()

        # Save both formats
        df.to_csv(f"{CONFIG['output_dir']}/WHO_GHO_Malaria_Raw.csv", index=False)
        pivot_df.to_csv(f"{CONFIG['output_dir']}/WHO_GHO_Malaria_Pivot.csv", index=False)

        print(f"\n   ✅ Total records collected: {len(df)}")
        print(f"   ✅ Unique country-years: {len(pivot_df)}")
        print(f"   📁 Saved to: {CONFIG['output_dir']}/")

        return pivot_df
    else:
        print("\n   ❌ No WHO GHO data collected")
        return None

# -----------------------------------------------------------------------------
# 3.3 NASA POWER - Climate Data
# -----------------------------------------------------------------------------

def download_nasa_climate_data():
    """
    Download climate data from NASA POWER API for all major cities

    Returns:
        DataFrame: Daily climate data for all cities
    """

    print("\n📥 Downloading NASA POWER climate data...")

    base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"

    start_date = f"{CONFIG['start_year']}0101"
    end_date = f"{CONFIG['end_year']}1231"

    all_climate_data = []

    for city, coords in CITIES.items():
        print(f"\n   Processing: {city}, {coords['country']}")

        params = {
            'latitude': coords['lat'],
            'longitude': coords['lon'],
            'start': start_date,
            'end': end_date,
            'community': 'AG',
            'parameters': 'T2M,T2M_MAX,T2M_MIN,PRECTOTCORR,RH2M',
            'format': 'JSON',
            'user': 'anonymous'
        }

        try:
            response = requests.get(base_url, params=params, timeout=120)
            response.raise_for_status()

            data = response.json()

            if 'properties' in data and 'parameter' in data['properties']:
                params_data = data['properties']['parameter']
                dates = list(params_data['T2M'].keys())

                for date in dates:
                    all_climate_data.append({
                        'City': city,
                        'Country': coords['country'],
                        'Latitude': coords['lat'],
                        'Longitude': coords['lon'],
                        'Date': pd.to_datetime(date, format='%Y%m%d'),
                        'Temperature_Avg_C': params_data['T2M'].get(date),
                        'Temperature_Max_C': params_data['T2M_MAX'].get(date),
                        'Temperature_Min_C': params_data['T2M_MIN'].get(date),
                        'Precipitation_mm': params_data['PRECTOTCORR'].get(date),
                        'Humidity_Percent': params_data['RH2M'].get(date)
                    })

                print(f"      ✅ Downloaded {len(dates)} days of data")
            else:
                print(f"      ❌ Invalid response structure")

        except Exception as e:
            print(f"      ❌ Error: {e}")

        time.sleep(2)  # Rate limiting

    if all_climate_data:
        df = pd.DataFrame(all_climate_data)

        # Replace -999 (missing values) with NaN
        numeric_columns = ['Temperature_Avg_C', 'Temperature_Max_C', 'Temperature_Min_C',
                          'Precipitation_mm', 'Humidity_Percent']
        for col in numeric_columns:
            df[col] = df[col].replace(-999, pd.NA)

        # Save daily data
        daily_file = f"{CONFIG['output_dir']}/NASA_Climate_Daily.csv"
        df.to_csv(daily_file, index=False)

        print(f"\n   ✅ Total climate records: {len(df)}")
        print(f"   📁 Saved to: {daily_file}")

        return df
    else:
        print("\n   ❌ No climate data collected")
        return None

def aggregate_climate_to_monthly(daily_data):
    """
    Aggregate daily climate data to monthly averages

    Args:
        daily_data (DataFrame): Daily climate data

    Returns:
        DataFrame: Monthly aggregated climate data
    """

    print("\n📊 Aggregating climate data to monthly...")

    df = daily_data.copy()
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    # Aggregate by country, year, and month
    monthly = df.groupby(['Country', 'Year', 'Month']).agg({
        'Temperature_Avg_C': 'mean',
        'Temperature_Max_C': 'max',
        'Temperature_Min_C': 'min',
        'Precipitation_mm': 'sum',  # Total precipitation
        'Humidity_Percent': 'mean'
    }).reset_index()

    # Create date column
    monthly['Date'] = pd.to_datetime(monthly[['Year', 'Month']].assign(Day=1))

    # Save
    monthly_file = f"{CONFIG['output_dir']}/NASA_Climate_Monthly.csv"
    monthly.to_csv(monthly_file, index=False)

    print(f"   ✅ Monthly records: {len(monthly)}")
    print(f"   📁 Saved to: {monthly_file}")

    return monthly

# -----------------------------------------------------------------------------
# 3.4 World Bank - Socioeconomic Indicators
# -----------------------------------------------------------------------------

def download_worldbank_data():
    """
    Download socioeconomic indicators from World Bank API

    Returns:
        DataFrame: World Bank indicators for target countries
    """

    print("\n📥 Downloading World Bank socioeconomic indicators...")

    base_url = "https://api.worldbank.org/v2/country"

    # Key indicators
    indicators = {
        'SP.POP.TOTL': 'Total_Population',
        'SP.RUR.TOTL.ZS': 'Rural_Population_Percent',
        'SP.URB.TOTL.IN.ZS': 'Urban_Population_Percent',
        'NY.GDP.PCAP.CD': 'GDP_per_Capita_USD',
        'SH.H2O.BASW.ZS': 'Basic_Water_Access_Percent',
        'SH.STA.BASS.ZS': 'Basic_Sanitation_Access_Percent',
        'SH.DYN.MORT': 'Under5_Mortality_per_1000',
        'SH.MED.BEDS.ZS': 'Hospital_Beds_per_1000'
    }

    all_data = []

    for country_code in CONFIG['countries_iso']:
        for indicator_code, indicator_name in indicators.items():
            print(f"   Processing: {indicator_name} for {CONFIG['country_names'][country_code]}")

            url = f"{base_url}/{country_code}/indicator/{indicator_code}"
            params = {
                'date': f"{CONFIG['start_year']}:{CONFIG['end_year']}",
                'format': 'json',
                'per_page': 500
            }

            try:
                response = requests.get(url, params=params, timeout=30)

                if response.status_code == 200:
                    data = response.json()

                    if len(data) > 1 and data[1]:
                        for record in data[1]:
                            if record['value'] is not None:
                                all_data.append({
                                    'Country': CONFIG['country_names'][country_code],
                                    'Country_Code': country_code,
                                    'Year': int(record['date']),
                                    'Indicator': indicator_name,
                                    'Value': record['value']
                                })
                        print(f"      ✅ Retrieved {len(data[1])} records")
                    else:
                        print(f"      ⚠️  No data available")
                else:
                    print(f"      ❌ HTTP {response.status_code}")

            except Exception as e:
                print(f"      ❌ Error: {e}")

            time.sleep(0.3)  # Rate limiting

    if all_data:
        df = pd.DataFrame(all_data)

        # Pivot to wide format
        pivot_df = df.pivot_table(
            index=['Country', 'Country_Code', 'Year'],
            columns='Indicator',
            values='Value',
            aggfunc='first'
        ).reset_index()

        # Save
        pivot_file = f"{CONFIG['output_dir']}/WorldBank_Indicators.csv"
        pivot_df.to_csv(pivot_file, index=False)

        print(f"\n   ✅ Total records: {len(df)}")
        print(f"   ✅ Country-years: {len(pivot_df)}")
        print(f"   📁 Saved to: {pivot_file}")

        return pivot_df
    else:
        print("\n   ❌ No World Bank data collected")
        return None

print("✅ All data collection functions defined!\n")

# =============================================================================
# SECTION 4: DATA MERGING & INTEGRATION
# =============================================================================

print("=" * 80)
print("SECTION 4: Defining Data Merging Functions")
print("=" * 80)

def create_master_dataset():
    """
    Merge all collected data sources into a single master dataset

    Returns:
        DataFrame: Integrated master dataset
    """

    print("\n🔗 Creating master dataset by merging all sources...")

    # Initialize master dataframe
    master_df = None

    # ---------------------------------------------------------------------
    # Load OWID data (primary source for disease data)
    # ---------------------------------------------------------------------
    try:
        owid_deaths = pd.read_csv(f"{CONFIG['output_dir']}/OWID_Malaria_Deaths.csv")
        owid_incidence = pd.read_csv(f"{CONFIG['output_dir']}/OWID_Malaria_Incidence.csv")

        # Start with deaths data
        master_df = owid_deaths[['Country', 'Year']].drop_duplicates()

        # Merge death columns
        death_cols = [col for col in owid_deaths.columns if col not in ['Country', 'Year', 'Entity', 'Code']]
        for col in death_cols:
            master_df = master_df.merge(
                owid_deaths[['Country', 'Year', col]],
                on=['Country', 'Year'],
                how='left',
                suffixes=('', '_OWID_Deaths')
            )

        # Merge incidence columns
        inc_cols = [col for col in owid_incidence.columns if col not in ['Country', 'Year', 'Entity', 'Code']]
        for col in inc_cols:
            master_df = master_df.merge(
                owid_incidence[['Country', 'Year', col]],
                on=['Country', 'Year'],
                how='left',
                suffixes=('', '_OWID_Inc')
            )

        print("   ✅ Merged OWID malaria data")

    except FileNotFoundError:
        print("   ⚠️  OWID data not found, skipping...")

    # ---------------------------------------------------------------------
    # Merge WHO GHO data
    # ---------------------------------------------------------------------
    try:
        who_gho = pd.read_csv(f"{CONFIG['output_dir']}/WHO_GHO_Malaria_Pivot.csv")

        if master_df is None:
            master_df = who_gho[['Country', 'Year']].drop_duplicates()

        who_cols = [col for col in who_gho.columns if col not in ['Country', 'Year', 'Country_Code']]

        master_df = master_df.merge(
            who_gho,
            on=['Country', 'Year'],
            how='outer',
            suffixes=('', '_WHO')
        )

        print("   ✅ Merged WHO GHO data")

    except FileNotFoundError:
        print("   ⚠️  WHO GHO data not found, skipping...")

    # ---------------------------------------------------------------------
    # Merge Climate data
    # ---------------------------------------------------------------------
    try:
        climate = pd.read_csv(f"{CONFIG['output_dir']}/NASA_Climate_Monthly.csv")

        # Aggregate by country and year
        climate_yearly = climate.groupby(['Country', 'Year']).agg({
            'Temperature_Avg_C': 'mean',
            'Temperature_Max_C': 'max',
            'Temperature_Min_C': 'min',
            'Precipitation_mm': 'sum',
            'Humidity_Percent': 'mean'
        }).reset_index()

        master_df = master_df.merge(
            climate_yearly,
            on=['Country', 'Year'],
            how='left'
        )

        print("   ✅ Merged NASA climate data")

    except FileNotFoundError:
        print("   ⚠️  Climate data not found, skipping...")

    # ---------------------------------------------------------------------
    # Merge World Bank data
    # ---------------------------------------------------------------------
    try:
        worldbank = pd.read_csv(f"{CONFIG['output_dir']}/WorldBank_Indicators.csv")

        wb_cols = [col for col in worldbank.columns if col not in ['Country', 'Year', 'Country_Code']]

        master_df = master_df.merge(
            worldbank[['Country', 'Year'] + wb_cols],
            on=['Country', 'Year'],
            how='left'
        )

        print("   ✅ Merged World Bank indicators")

    except FileNotFoundError:
        print("   ⚠️  World Bank data not found, skipping...")

    # ---------------------------------------------------------------------
    # Feature Engineering
    # ---------------------------------------------------------------------
    if master_df is not None:
        print("\n🔧 Performing feature engineering...")

        # Sort by country and year
        master_df = master_df.sort_values(['Country', 'Year']).reset_index(drop=True)

        # Create temporal features
        master_df['Year_Since_Start'] = master_df['Year'] - CONFIG['start_year']
        master_df['Year_Normalized'] = (master_df['Year'] - CONFIG['start_year']) / (CONFIG['end_year'] - CONFIG['start_year'])

        # Create year-over-year change features for key indicators
        numeric_cols = master_df.select_dtypes(include=[np.number]).columns
        numeric_cols = [col for col in numeric_cols if col not in ['Year', 'Year_Since_Start', 'Year_Normalized']]

        for col in numeric_cols:
            master_df[f'{col}_YoY_Change'] = master_df.groupby('Country')[col].pct_change()
            master_df[f'{col}_YoY_Diff'] = master_df.groupby('Country')[col].diff()

        # Remove columns with > 80% missing values
        threshold = 0.8
        missing_pct = master_df.isnull().sum() / len(master_df)
        cols_to_keep = missing_pct[missing_pct < threshold].index.tolist()

        dropped_cols = [col for col in master_df.columns if col not in cols_to_keep]
        if dropped_cols:
            print(f"   ⚠️  Dropped {len(dropped_cols)} columns with >80% missing values")

        master_df = master_df[cols_to_keep]

        # Save master dataset
        master_file = f"{CONFIG['output_dir']}/MASTER_Dataset.csv"
        master_df.to_csv(master_file, index=False)

        print(f"\n   ✅ Master dataset created!")
        print(f"   📊 Shape: {master_df.shape}")
        print(f"   📊 Countries: {master_df['Country'].unique().tolist()}")
        print(f"   📊 Years: {master_df['Year'].min()} - {master_df['Year'].max()}")
        print(f"   📁 Saved to: {master_file}")

        return master_df
    else:
        print("\n   ❌ Could not create master dataset - no data sources available")
        return None

print("✅ Data merging functions defined!\n")

# =============================================================================
# SECTION 5: DATA QUALITY & VISUALIZATION
# =============================================================================

print("=" * 80)
print("SECTION 5: Defining Quality Check & Visualization Functions")
print("=" * 80)

def generate_data_quality_report(master_df):
    """
    Generate comprehensive data quality report

    Args:
        master_df (DataFrame): Master dataset
    """

    print("\n📊 Generating data quality report...")

    # Basic statistics
    report = {
        'Total Records': len(master_df),
        'Countries': master_df['Country'].nunique(),
        'Years': master_df['Year'].nunique(),
        'Total Features': len(master_df.columns),
        'Numeric Features': len(master_df.select_dtypes(include=[np.number]).columns),
        'Object Features': len(master_df.select_dtypes(include=['object']).columns)
    }

    print("\n   📋 Dataset Overview:")
    for key, value in report.items():
        print(f"      {key}: {value}")

    # Missing data analysis
    missing_data = pd.DataFrame({
        'Column': master_df.columns,
        'Missing_Count': master_df.isnull().sum(),
        'Missing_Percent': (master_df.isnull().sum() / len(master_df) * 100).round(2)
    }).sort_values('Missing_Percent', ascending=False)

    missing_file = f"{CONFIG['output_dir']}/Data_Quality_Report.csv"
    missing_data.to_csv(missing_file, index=False)

    print(f"\n   📁 Quality report saved to: {missing_file}")

    # Print top 10 columns with missing data
    if missing_data['Missing_Percent'].max() > 0:
        print("\n   ⚠️  Top 10 columns with missing data:")
        top_missing = missing_data[missing_data['Missing_Percent'] > 0].head(10)
        for _, row in top_missing.iterrows():
            print(f"      {row['Column']}: {row['Missing_Percent']}%")
    else:
        print("\n   ✅ No missing data detected!")

    return missing_data

def create_eda_visualizations(master_df):
    """
    Create exploratory data analysis visualizations

    Args:
        master_df (DataFrame): Master dataset
    """

    print("\n📊 Creating exploratory data analysis visualizations...")

    # Set style
    sns.set_style("whitegrid")
    plt.rcParams['figure.figsize'] = (16, 12)

    # -------------------------------------------------------------------------
    # Figure 1: Temporal Trends
    # -------------------------------------------------------------------------
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))

    # Find malaria-related columns
    malaria_cols = [col for col in master_df.columns
                   if any(x in col.lower() for x in ['malaria', 'death', 'case', 'incidence'])]

    if malaria_cols:
        # Plot 1: Malaria cases/deaths over time
        for country in master_df['Country'].unique():
            country_data = master_df[master_df['Country'] == country]
            if malaria_cols[0] in country_data.columns:
                axes[0, 0].plot(country_data['Year'], country_data[malaria_cols[0]],
                              marker='o', label=country, linewidth=2)

        axes[0, 0].set_title('Malaria Indicator Trends by Country', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Year')
        axes[0, 0].set_ylabel(malaria_cols[0])
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

    # Plot 2: Temperature trends
    if 'Temperature_Avg_C' in master_df.columns:
        for country in master_df['Country'].unique():
            country_data = master_df[master_df['Country'] == country]
            axes[0, 1].plot(country_data['Year'], country_data['Temperature_Avg_C'],
                          marker='s', label=country, linewidth=2)

        axes[0, 1].set_title('Average Temperature Trends', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Year')
        axes[0, 1].set_ylabel('Temperature (°C)')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

    # Plot 3: Precipitation patterns
    if 'Precipitation_mm' in master_df.columns:
        for country in master_df['Country'].unique():
            country_data = master_df[master_df['Country'] == country]
            axes[1, 0].plot(country_data['Year'], country_data['Precipitation_mm'],
                          marker='^', label=country, linewidth=2)

        axes[1, 0].set_title('Annual Precipitation Patterns', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Year')
        axes[1, 0].set_ylabel('Precipitation (mm)')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

    # Plot 4: Population growth
    pop_cols = [col for col in master_df.columns if 'population' in col.lower()]
    if pop_cols:
        for country in master_df['Country'].unique():
            country_data = master_df[master_df['Country'] == country]
            if pop_cols[0] in country_data.columns:
                axes[1, 1].plot(country_data['Year'], country_data[pop_cols[0]],
                              marker='d', label=country, linewidth=2)

        axes[1, 1].set_title('Population Growth Trends', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Year')
        axes[1, 1].set_ylabel('Population')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    trends_file = f"{CONFIG['plots_dir']}/01_Temporal_Trends.png"
    plt.savefig(trends_file, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   ✅ Saved: {trends_file}")

    # -------------------------------------------------------------------------
    # Figure 2: Correlation Matrix
    # -------------------------------------------------------------------------
    numeric_df = master_df.select_dtypes(include=[np.number])

    # Remove YoY change columns for cleaner correlation matrix
    numeric_df = numeric_df[[col for col in numeric_df.columns
                            if not any(x in col for x in ['YoY_Change', 'YoY_Diff', 'Year_Since_Start'])]]

    if len(numeric_df.columns) > 1:
        plt.figure(figsize=(20, 16))
        corr_matrix = numeric_df.corr()

        sns.heatmap(corr_matrix,
                   annot=False,
                   cmap='coolwarm',
                   center=0,
                   linewidths=0.5,
                   cbar_kws={'label': 'Correlation Coefficient'})

        plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()

        corr_file = f"{CONFIG['plots_dir']}/02_Correlation_Matrix.png"
        plt.savefig(corr_file, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅ Saved: {corr_file}")

    # -------------------------------------------------------------------------
    # Figure 3: Country Comparisons
    # -------------------------------------------------------------------------
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    countries = master_df['Country'].unique()
    colors = sns.color_palette("husl", len(countries))

    # Plot 1: Malaria distribution by country
    if malaria_cols:
        for idx, country in enumerate(countries):
            country_data = master_df[master_df['Country'] == country][malaria_cols[0]].dropna()
            if len(country_data) > 0:
                axes[0, 0].hist(country_data, alpha=0.6, label=country,
                              color=colors[idx], bins=12, edgecolor='black')

        axes[0, 0].set_title('Distribution of Malaria Cases by Country',
                            fontsize=12, fontweight='bold')
        axes[0, 0].set_xlabel('Cases')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].legend()

    # Plot 2: Temperature distribution by country
    if 'Temperature_Avg_C' in master_df.columns:
        temp_data = [master_df[master_df['Country'] == country]['Temperature_Avg_C'].dropna()
                     for country in countries]

        bp = axes[0, 1].boxplot(temp_data, labels=countries, patch_artist=True)

        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)

        axes[0, 1].set_title('Temperature Distribution by Country',
                            fontsize=12, fontweight='bold')
        axes[0, 1].set_xlabel('Country')
        axes[0, 1].set_ylabel('Temperature (°C)')
        axes[0, 1].tick_params(axis='x', rotation=45)

    # Plot 3: Precipitation distribution by country
    if 'Precipitation_mm' in master_df.columns:
        precip_data = [master_df[master_df['Country'] == country]['Precipitation_mm'].dropna()
                      for country in countries]

        bp = axes[1, 0].boxplot(precip_data, labels=countries, patch_artist=True)

        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)

        axes[1, 0].set_title('Precipitation Distribution by Country',
                            fontsize=12, fontweight='bold')
        axes[1, 0].set_xlabel('Country')
        axes[1, 0].set_ylabel('Precipitation (mm)')
        axes[1, 0].tick_params(axis='x', rotation=45)

    # Plot 4: Average malaria by country (bar chart)
    if malaria_cols:
        country_means = master_df.groupby('Country')[malaria_cols[0]].mean().sort_values()

        bars = axes[1, 1].barh(range(len(country_means)), country_means.values, color=colors)
        axes[1, 1].set_yticks(range(len(country_means)))
        axes[1, 1].set_yticklabels(country_means.index)

        axes[1, 1].set_title(f'Average {malaria_cols[0]} by Country',
                            fontsize=12, fontweight='bold')
        axes[1, 1].set_xlabel('Average Value')

    plt.tight_layout()
    comparison_file = f"{CONFIG['plots_dir']}/03_Country_Comparisons.png"
    plt.savefig(comparison_file, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   ✅ Saved: {comparison_file}")

    # -------------------------------------------------------------------------
    # Figure 4: Key Statistics Summary
    # -------------------------------------------------------------------------
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Plot 1: Data completeness by country
    completeness = []
    for country in countries:
        country_data = master_df[master_df['Country'] == country]
        complete_pct = (1 - country_data.isnull().sum().sum() /
                       (len(country_data) * len(country_data.columns))) * 100
        completeness.append(complete_pct)

    axes[0, 0].bar(countries, completeness, color=colors, edgecolor='black')
    axes[0, 0].set_title('Data Completeness by Country', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Country')
    axes[0, 0].set_ylabel('Completeness (%)')
    axes[0, 0].set_ylim([0, 100])
    axes[0, 0].tick_params(axis='x', rotation=45)

    # Add value labels on bars
    for i, v in enumerate(completeness):
        axes[0, 0].text(i, v + 2, f'{v:.1f}%', ha='center', va='bottom', fontweight='bold')

    # Plot 2: Number of records per country
    records_per_country = master_df['Country'].value_counts().sort_index()

    axes[0, 1].bar(records_per_country.index, records_per_country.values,
                   color=colors, edgecolor='black')
    axes[0, 1].set_title('Number of Records by Country', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Country')
    axes[0, 1].set_ylabel('Number of Records')
    axes[0, 1].tick_params(axis='x', rotation=45)

    for i, v in enumerate(records_per_country.values):
        axes[0, 1].text(i, v + 0.5, str(v), ha='center', va='bottom', fontweight='bold')

    # Plot 3: Feature category distribution
    feature_categories = {
        'Disease': len([col for col in master_df.columns
                       if any(x in col.lower() for x in ['malaria', 'death', 'case', 'disease'])]),
        'Climate': len([col for col in master_df.columns
                       if any(x in col.lower() for x in ['temperature', 'precipitation', 'humidity'])]),
        'Socioeconomic': len([col for col in master_df.columns
                             if any(x in col.lower() for x in ['population', 'gdp', 'water', 'sanitation'])]),
        'Temporal': len([col for col in master_df.columns
                        if any(x in col.lower() for x in ['year', 'date'])]),
        'Other': len(master_df.columns)
    }

    # Adjust 'Other' to not double count
    feature_categories['Other'] = feature_categories['Other'] - sum([v for k, v in feature_categories.items() if k != 'Other'])

    axes[1, 0].pie(feature_categories.values(),
                   labels=feature_categories.keys(),
                   autopct='%1.1f%%',
                   startangle=90,
                   colors=sns.color_palette("pastel"))
    axes[1, 0].set_title('Feature Distribution by Category', fontsize=12, fontweight='bold')

    # Plot 4: Missing data summary
    missing_by_feature_type = {
        'Disease': master_df[[col for col in master_df.columns
                             if any(x in col.lower() for x in ['malaria', 'death', 'case'])]].isnull().sum().sum(),
        'Climate': master_df[[col for col in master_df.columns
                             if any(x in col.lower() for x in ['temperature', 'precipitation', 'humidity'])]].isnull().sum().sum(),
        'Socioeconomic': master_df[[col for col in master_df.columns
                                   if any(x in col.lower() for x in ['population', 'gdp', 'water', 'sanitation'])]].isnull().sum().sum()
    }

    axes[1, 1].bar(missing_by_feature_type.keys(), missing_by_feature_type.values(),
                  color=sns.color_palette("pastel"), edgecolor='black')
    axes[1, 1].set_title('Missing Values by Feature Category', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Feature Category')
    axes[1, 1].set_ylabel('Missing Values Count')
    axes[1, 1].tick_params(axis='x', rotation=45)

    for i, v in enumerate(missing_by_feature_type.values()):
        axes[1, 1].text(i, v + max(missing_by_feature_type.values())*0.02,
                      str(v), ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    summary_file = f"{CONFIG['plots_dir']}/04_Summary_Statistics.png"
    plt.savefig(summary_file, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"   ✅ Saved: {summary_file}")

    print("\n   ✅ All visualizations created successfully!")

def generate_descriptive_statistics(master_df):
    """
    Generate and save descriptive statistics

    Args:
        master_df (DataFrame): Master dataset
    """

    print("\n📊 Generating descriptive statistics...")

    # Overall statistics
    desc_stats = master_df.describe().T
    desc_stats_file = f"{CONFIG['output_dir']}/Descriptive_Statistics_Overall.csv"
    desc_stats.to_csv(desc_stats_file)
    print(f"   ✅ Saved overall statistics to: {desc_stats_file}")

    # Statistics by country
    numeric_cols = master_df.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col not in ['Year', 'Year_Since_Start', 'Year_Normalized']]

    country_stats_list = []

    for country in master_df['Country'].unique():
        country_data = master_df[master_df['Country'] == country][numeric_cols]
        stats = country_data.describe().T
        stats['Country'] = country
        country_stats_list.append(stats)

    country_stats = pd.concat(country_stats_list)
    country_stats_file = f"{CONFIG['output_dir']}/Descriptive_Statistics_By_Country.csv"
    country_stats.to_csv(country_stats_file)
    print(f"   ✅ Saved country statistics to: {country_stats_file}")

print("✅ Quality check and visualization functions defined!\n")

# =============================================================================
# SECTION 6: MASTER EXECUTION PIPELINE
# =============================================================================

print("=" * 80)
print("SECTION 6: Master Execution Pipeline")
print("=" * 80)

def execute_complete_pipeline():
    """
    Execute the complete data collection and processing pipeline

    Returns:
        dict: Results summary
    """

    start_time = datetime.now()

    print("\n" + "=" * 80)
    print("🚀 STARTING COMPLETE DATA COLLECTION PIPELINE")
    print("=" * 80)
    print(f"Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

    results = {
        'start_time': start_time,
        'data_sources': {},
        'files_created': []
    }

    # -------------------------------------------------------------------------
    # STEP 1: Download OWID Malaria Data
    # -------------------------------------------------------------------------
    print("\n" + "=" * 80)
    print("STEP 1/6: Downloading Our World in Data (OWID) Malaria Data")
    print("=" * 80)

    try:
        owid_data = download_owid_malaria_data()
        results['data_sources']['owid'] = 'SUCCESS' if owid_data else 'FAILED'
    except Exception as e:
        print(f"❌ OWID download failed: {e}")
        results['data_sources']['owid'] = 'FAILED'

    # -------------------------------------------------------------------------
    # STEP 2: Download WHO GHO Data
    # -------------------------------------------------------------------------
    print("\n" + "=" * 80)
    print("STEP 2/6: Downloading WHO Global Health Observatory (GHO) Data")
    print("=" * 80)

    try:
        who_data = download_who_gho_data()
        results['data_sources']['who_gho'] = 'SUCCESS' if who_data is not None else 'FAILED'
    except Exception as e:
        print(f"❌ WHO GHO download failed: {e}")
        results['data_sources']['who_gho'] = 'FAILED'

    # -------------------------------------------------------------------------
    # STEP 3: Download NASA Climate Data
    # -------------------------------------------------------------------------
    print("\n" + "=" * 80)
    print("STEP 3/6: Downloading NASA POWER Climate Data")
    print("=" * 80)

    try:
        climate_daily = download_nasa_climate_data()
        if climate_daily is not None:
            climate_monthly = aggregate_climate_to_monthly(climate_daily)
            results['data_sources']['nasa_climate'] = 'SUCCESS'
        else:
            results['data_sources']['nasa_climate'] = 'FAILED'
    except Exception as e:
        print(f"❌ NASA climate download failed: {e}")
        results['data_sources']['nasa_climate'] = 'FAILED'

    # -------------------------------------------------------------------------
    # STEP 4: Download World Bank Data
    # -------------------------------------------------------------------------
    print("\n" + "=" * 80)
    print("STEP 4/6: Downloading World Bank Socioeconomic Indicators")
    print("=" * 80)

    try:
        wb_data = download_worldbank_data()
        results['data_sources']['worldbank'] = 'SUCCESS' if wb_data is not None else 'FAILED'
    except Exception as e:
        print(f"❌ World Bank download failed: {e}")
        results['data_sources']['worldbank'] = 'FAILED'

    # -------------------------------------------------------------------------
    # STEP 5: Create Master Dataset
    # -------------------------------------------------------------------------
    print("\n" + "=" * 80)
    print("STEP 5/6: Creating Master Integrated Dataset")
    print("=" * 80)

    try:
        master_df = create_master_dataset()
        if master_df is not None:
            results['data_sources']['master_dataset'] = 'SUCCESS'
            results['master_df'] = master_df
        else:
            results['data_sources']['master_dataset'] = 'FAILED'
            print("⚠️  WARNING: Master dataset creation failed!")
    except Exception as e:
        print(f"❌ Master dataset creation failed: {e}")
        results['data_sources']['master_dataset'] = 'FAILED'

    # -------------------------------------------------------------------------
    # STEP 6: Generate Analysis & Visualizations
    # -------------------------------------------------------------------------
    print("\n" + "=" * 80)
    print("STEP 6/6: Generating Quality Reports & Visualizations")
    print("=" * 80)

    if 'master_df' in results and results['master_df'] is not None:
        try:
            # Quality report
            quality_report = generate_data_quality_report(results['master_df'])

            # Descriptive statistics
            generate_descriptive_statistics(results['master_df'])

            # Visualizations
            create_eda_visualizations(results['master_df'])

            results['data_sources']['analysis'] = 'SUCCESS'
        except Exception as e:
            print(f"❌ Analysis generation failed: {e}")
            results['data_sources']['analysis'] = 'FAILED'
    else:
        print("⚠️  Skipping analysis - no master dataset available")
        results['data_sources']['analysis'] = 'SKIPPED'

    # -------------------------------------------------------------------------
    # FINAL SUMMARY
    # -------------------------------------------------------------------------
    end_time = datetime.now()
    duration = end_time - start_time

    print("\n" + "=" * 80)
    print("📊 PIPELINE EXECUTION SUMMARY")
    print("=" * 80)

    print(f"\n⏱️  Execution Time: {duration}")
    print(f"   Started: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"   Ended: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")

    print("\n📦 Data Source Status:")
    for source, status in results['data_sources'].items():
        icon = "✅" if status == "SUCCESS" else "❌" if status == "FAILED" else "⚠️"
        print(f"   {icon} {source.upper()}: {status}")

    # List files created
    print("\n📁 Files Created:")

    if os.path.exists(CONFIG['output_dir']):
        data_files = [f for f in os.listdir(CONFIG['output_dir']) if f.endswith('.csv')]
        for file in sorted(data_files):
            file_path = os.path.join(CONFIG['output_dir'], file)
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
            print(f"   ✅ {file} ({file_size:.2f} MB)")
            results['files_created'].append(file)

    if os.path.exists(CONFIG['plots_dir']):
        plot_files = [f for f in os.listdir(CONFIG['plots_dir']) if f.endswith('.png')]
        for file in sorted(plot_files):
            print(f"   ✅ {file}")
            results['files_created'].append(file)

    # Success rate
    success_count = sum(1 for status in results['data_sources'].values() if status == 'SUCCESS')
    total_count = len([s for s in results['data_sources'].values() if s != 'SKIPPED'])
    success_rate = (success_count / total_count * 100) if total_count > 0 else 0

    print(f"\n📈 Success Rate: {success_count}/{total_count} ({success_rate:.1f}%)")

    if 'master_df' in results and results['master_df'] is not None:
        print("\n✅✅✅ PIPELINE COMPLETED SUCCESSFULLY!")
        print(f"\n🎯 Master Dataset Ready for Machine Learning!")
        print(f"   Location: {CONFIG['output_dir']}/MASTER_Dataset.csv")
        print(f"   Shape: {results['master_df'].shape}")
        print(f"   Features: {len(results['master_df'].columns)}")
        print(f"   Records: {len(results['master_df'])}")
    else:
        print("\n⚠️  PIPELINE COMPLETED WITH WARNINGS")
        print("   Some data sources failed. Check logs above for details.")

    print("\n" + "=" * 80)

    results['end_time'] = end_time
    results['duration'] = duration
    results['success_rate'] = success_rate

    return results

print("✅ Master pipeline function defined!\n")

# =============================================================================
# SECTION 7: EXECUTION
# =============================================================================

print("=" * 80)
print("SECTION 7: Execute Pipeline")
print("=" * 80)
print("\n🚀 Ready to execute the complete data collection pipeline!")
print("\nThis will:")
print("   1. Download malaria data from OWID")
print("   2. Download WHO GHO malaria indicators")
print("   3. Download NASA climate data for 10 cities")
print("   4. Download World Bank socioeconomic indicators")
print("   5. Merge all data sources into master dataset")
print("   6. Generate quality reports and visualizations")
print("\n⏱️  Estimated time: 15-30 minutes")
print("\n" + "=" * 80)

# Execute the pipeline
results = execute_complete_pipeline()

# =============================================================================
# SECTION 8: POST-EXECUTION ANALYSIS
# =============================================================================

print("\n" + "=" * 80)
print("SECTION 8: Post-Execution Data Preview")
print("=" * 80)

if 'master_df' in results and results['master_df'] is not None:
    master_df = results['master_df']

    print("\n📊 Master Dataset Preview:")
    print("\nFirst 5 rows:")
    display(master_df.head())

    print("\nDataset Info:")
    print(f"   Shape: {master_df.shape}")
    print(f"   Memory Usage: {master_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    print("\nColumn Names:")
    for i, col in enumerate(master_df.columns, 1):
        print(f"   {i}. {col}")

    print("\nData Types:")
    print(master_df.dtypes.value_counts())

    print("\nMissing Data Summary:")
    missing_summary = master_df.isnull().sum()
    missing_pct = (missing_summary / len(master_df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing_Count': missing_summary,
        'Missing_Percent': missing_pct
    }).sort_values('Missing_Percent', ascending=False).head(10)
    display(missing_df)

    print("\n✅ Data collection complete! Ready for machine learning modeling.")

else:
    print("\n⚠️  Master dataset not available. Please check error messages above.")

print("\n" + "=" * 80)
print("🎉 ALL DONE! Check the output folders for your data and visualizations.")
print("=" * 80)

In [ ]:
# After execution completes, run this cell to download all data

from google.colab import files
import shutil

# Create zip file with all results
shutil.make_archive('/content/Central_Africa_Disease_Data', 'zip', '/content/data')
shutil.make_archive('/content/Central_Africa_Plots', 'zip', '/content/plots')

# Download
files.download('/content/Central_Africa_Disease_Data.zip')
files.download('/content/Central_Africa_Plots.zip')

print("✅ Downloads started! Check your browser's download folder.")

print("\n✅ Data collection complete! Ready for machine learning modeling.")

print("\n" + "=" * 80)
print(" ALL DONE! Check the output folders for your data and visualizations.")
print("=" * 80)